In [1]:
import PySimpleGUI as sg
import time

# def setup():
# global pyaudio, wave, pathlib, threading, csv, librosa, pd, np, plt
import pyaudio, wave, pathlib, threading, csv, librosa
# import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# from PIL import Image
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.preprocessing import LabelEncoder, StandardScaler
# from sklearn.svm import SVC
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
# from xgboost import XGBClassifier

# import pickle
# import keras
# from keras import layers
# from keras.models import Sequential
import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.models import load_model 
model = load_model('ff_nn_v1_Ac85.h5')

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "output.wav"
# stop = 0
p = pyaudio.PyAudio()
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)


In [2]:

def predict(name,itr=0):
    global model
    #Doing prediction
    cmap = plt.get_cmap('inferno')
    plt.figure(figsize=(8,8))
    pathlib.Path(f'live_data/').mkdir(parents=True, exist_ok=True)
    # for filename in os.listdir(f'b_cry/{g}'):
    songname = name+'.wav'
    y, sr = librosa.load(songname, mono=True, duration=7)
    plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
    plt.axis('off');
    plt.savefig(f'live_data/{name}.png')
    plt.clf()

    header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
    for i in range(1, 21):
        header += f' mfcc{i}'
    header += ' label'
    header = header.split()

    file = open(f'{name}.csv', 'w', newline='')
    with file:
        writer = csv.writer(file)
        writer.writerow(header)
    y, sr = librosa.load(songname, mono=True, duration=5)
    rmse = librosa.feature.rms(y=y)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    to_append = f'{name}.wav {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'   
    for e in mfcc:
        to_append += f' {np.mean(e)}'
    file = open(f'{name}.csv', 'a', newline='')
    with file:
        writer = csv.writer(file)
        writer.writerow(to_append.split())
    live_data = pd.read_csv(f'{name}.csv')
    live_data = live_data.drop(['filename'],axis=1)
    Xnew = np.array([live_data.iloc[0][0:26]])
    ynew = model.predict_classes(Xnew)
    # if ynew==0:
    #     print(str(itr)+' - Cry')
    # else:
    #     print(str(itr)+' - No cry')
    print(str(y)+(' - Cry',' - No cry')[ynew])


In [3]:
def record():
    frames = []
    songname = "output"
    WAVE_OUTPUT_FILENAME = "output.wav"
    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    
    t2 = threading.Thread(target=predict, args=(songname, j))
    t2.start()

In [4]:
def close():
    """Close all open streams/files"""
    # global stream, p, wf
    stream.stop_stream()
    stream.close()
    p.terminate()
    # wf.close()

In [5]:
def make_window():
    sg.theme('Dark')
    sg.set_options(element_padding=(0, 0))

    layout = [[sg.Button('Start Recording', button_color=('white', 'black')),
            sg.Button('Stop', button_color=('gray50', 'black')),
            sg.Button('Reset', button_color=('white', '#9B0023')),
            sg.Button('Submit', button_color=('gray60', 'springgreen4')),
            sg.Button('Exit', button_color=('white', '#00406B'))],
            [sg.ProgressBar(5, orientation='h', size=(40, 10), key='-PROGRESS BAR-')]]

    window = sg.Window("Borderless Window",
                    layout,
                    default_element_size=(12, 1),
                    text_justification='r',
                    auto_size_text=False,
                    auto_size_buttons=False,
                    no_titlebar=True,
                    grab_anywhere=True,
                    default_button_element_size=(12, 1))
    return sg.Window('All Elements Demo', layout)


In [6]:
window = make_window()
while True:
    event, values = window.read()
    if event in (sg.WIN_CLOSED, 'Exit'):
        print('============ Event = ', event, ' ==============')
        print('-------- Values Dictionary (key=value) --------')
        for key in values:
            print(key, ' = ',values[key])
        close()
        break
    elif event == 'Start Recording':
        print("[LOG] Clicked Test Progress Bar!")
        record()

No editor has been configured in the global settings


KeyboardInterrupt: 